In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [25]:
train_user = pd.read_csv('../input/训练数据-ccf_first_round_user_shop_behavior.csv')
train_shop = pd.read_csv('../input/训练数据-ccf_first_round_shop_info.csv')
test = pd.read_csv('../input/AB榜测试集-evaluation_public.csv')

train_user.shape
train_user.head()
train_shop.shape
train_shop.head()
test.shape
test.head()

(1138015, 6)

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...


(8477, 6)

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,c_4,122.346736,31.833507,57,m_690
1,s_133,c_6,121.134362,31.197511,58,m_6587
2,s_251,c_38,121.000505,30.907667,34,m_5892
3,s_372,c_30,119.864982,26.659876,44,m_625
4,s_456,c_26,122.594243,31.581499,44,m_3839


(483931, 7)

,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos
0,118742,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,2017-09-06 13:10,118.191907,32.855858,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,2017-09-06 17:40,119.192110,32.424667,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,2017-09-03 12:10,120.612201,34.055249,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,2017-09-02 20:40,116.861989,40.326858,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [16]:
train = train_user.merge(train_shop[['shop_id', 'mall_id']], left_on='shop_id', right_on='shop_id', how='left')
# del train_user
# del train_shop

test = test.drop(['row_id'], axis=1)

In [20]:
train.columns

Index(['user_id', 'shop_id', 'time_stamp', 'longitude', 'latitude',
       'wifi_infos', 'mall_id', 'flag', 'full_order'],
      dtype='object')

In [17]:
train['flag'] = 1
train['full_order'] = train.index
mall_ids = set(train.mall_id)

train.shape
train.head()

(1138015, 9)

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos,mall_id,flag,full_order
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...,m_1409,1,0
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...,m_1409,1,1
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...,m_4079,1,2
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...,m_6587,1,3
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...,m_3005,1,4


In [6]:
train_new = []
train = train.as_matrix()

In [24]:
for i in tqdm(range(train.shape[0])):
    mall_id = train[i][10]
    train_new.append(train[i].tolist())
    m = pickle.load(open('./output/train_shop/' + mall_id + '.p', 'rb'))
    m = m.as_matrix()
    for j in range(1, 9):
        temp = train[i].tolist()
        temp[1] = m[j, i]
        temp[11] = 0
        train_new.append(temp)

  0%|          | 1251/1138015 [00:11<3:01:23, 104.45it/s]

KeyboardInterrupt: 

In [18]:
train_new = []

for mall_id in tqdm(mall_ids):
    m = pickle.load(open('./output/train_shop/' + mall_id + '.p', 'rb'))
    m = m.as_matrix()
    temp_train = train[train.mall_id == mall_id]
    temp_train = temp_train.as_matrix()
    for i in range(temp_train.shape[0]):
        train_new.append(temp_train[i].tolist())
        for j in range(1, 9):
            temp = temp_train[i].tolist()
            temp[1] = m[j, i]
            temp[7] = 0
            train_new.append(temp)

100%|██████████| 97/97 [00:32<00:00,  2.99it/s]


In [19]:
train_neg = pd.DataFrame(train_new)

train_neg.shape
train_neg.head(20)

(10242135, 9)

,0,1,2,3,4,5,6,7,8
0,u_23100,s_410892,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,1,123
1,u_23100,s_428507,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123
2,u_23100,s_3699611,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123
3,u_23100,s_8146,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123
4,u_23100,s_428193,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123
5,u_23100,s_3593857,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123
6,u_23100,s_428953,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123
7,u_23100,s_360429,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123
8,u_23100,s_140033,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123
9,u_23100,s_410892,2017-08-01 14:50,113.819782,34.948545,b_6937466|-47|true;b_6937467|-54|false;b_37960...,m_2270,1,124


In [23]:
train_neg.columns = ['user_id', 'shop_id', 'time_stamp', 'longitude', 'latitude', 'wifi_infos', 'mall_id', 'flag', 'full_order']

In [26]:
train_neg = train_neg.merge(train_shop.drop(['mall_id'], axis=1), left_on='shop_id', right_on='shop_id', how='left')

In [29]:
train_neg.shape
train_neg.head(20)

(10242135, 13)

,user_id,shop_id,time_stamp,longitude_x,latitude_x,wifi_infos,mall_id,flag,full_order,category_id,longitude_y,latitude_y,price
0,u_23100,s_410892,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,1,123,c_29,113.819891,34.948559,46
1,u_23100,s_428507,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123,c_132,113.819719,34.948597,57
2,u_23100,s_3699611,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123,c_27,113.819822,34.949240,39
3,u_23100,s_8146,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123,c_49,113.819720,34.948545,60
4,u_23100,s_428193,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123,c_28,113.819570,34.949528,47
5,u_23100,s_3593857,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123,c_29,113.819447,34.948581,47
6,u_23100,s_428953,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123,c_49,113.819409,34.949188,64
7,u_23100,s_360429,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123,c_30,113.819062,34.950881,44
8,u_23100,s_140033,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,m_2270,0,123,c_41,113.819415,34.948654,72
9,u_23100,s_410892,2017-08-01 14:50,113.819782,34.948545,b_6937466|-47|true;b_6937467|-54|false;b_37960...,m_2270,1,124,c_29,113.819891,34.948559,46


In [4]:
test_new = []
train_user = train_user.as_matrix()
test = test.as_matrix()

In [5]:
for i in tqdm(range(test.shape[0])):
    mall_id = test[i][1]
    m = pickle.load(open('./output/test_user_neg10/' + mall_id + '.p', 'rb'))
    m.columns = m.iloc[0]
    for j in range(1, 6):
        temp = test[i].tolist()
        temp.append(train_user[m.loc[:, 1138015 + i][j]][1])
        test_new.append(temp.copy())

100%|██████████| 483931/483931 [23:32<00:00, 342.50it/s]


In [ ]:
train_new = []

for mall_id in tqdm(mall_ids):
    m = pickle.load(open('./output/test_user_neg10/' + mall_id + '.p', 'rb'))
    m = m.as_matrix()
    temp_train = train[train.mall_id == mall_id]
    temp_train = temp_train.as_matrix()
    for i in range(temp_train.shape[0]):
        train_new.append(temp_train[i].tolist())
        for j in range(1, 9):
            temp = temp_train[i].tolist()
            temp[1] = m[j, i]
            temp[11] = 0
            train_new.append(temp)

In [6]:
test_neg = pd.DataFrame(test_new)

test_neg.shape
test_neg.head()

(2419655, 7)

,0,1,2,3,4,5,6
0,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...,s_3683488
1,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...,s_147921
2,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...,s_312218
3,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...,s_10425
4,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...,s_3997599


In [ ]:
train_neg.to_csv('./output/train_neg9.csv', index=False)

In [7]:
test_neg.to_csv('./output/test_neg5.csv', index=False, header=['user_id', 'mall_id', 'time_stamp', 'longitude', 'latitude', 'wifi_infos', 'shop_id'])